In [1]:
import torch

## 1. 张量计算
### 1.1. 定义张量

In [2]:
# 创建一个尺寸为（5，3）的二阶张量
x = torch.rand(5, 3)
x

tensor([[0.9303, 0.4382, 0.7336],
        [0.0945, 0.5926, 0.4089],
        [0.3437, 0.1147, 0.2174],
        [0.7214, 0.4662, 0.1993],
        [0.2085, 0.2368, 0.6694]])

In [3]:
# 创建一个尺寸为（5, 3），内容全是1的张量
y = torch.ones(5, 3)
y

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [4]:
# 创建一个三维的张量，尺寸为（2,5,3），内容全是0
z = torch.zeros(2, 5, 3)
z

tensor([[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]])

### 1.2. 访问张量

In [5]:
# 访问张量z中的第一个元素
z[0]

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [6]:
# 访问张量x中第2行第3列的数字
x[1, 2]

tensor(0.4089)

In [7]:
# 还可以使用切片的方法来访问张量，访问张量x中第3列的全部元素
x[:, 2]

tensor([0.7336, 0.4089, 0.2174, 0.1993, 0.6694])

### 1.3. 张量的运算

In [8]:
# 相加
z = x + y
z

tensor([[1.9303, 1.4382, 1.7336],
        [1.0945, 1.5926, 1.4089],
        [1.3437, 1.1147, 1.2174],
        [1.7214, 1.4662, 1.1993],
        [1.2085, 1.2368, 1.6694]])

In [9]:
# 两个二维张量的矩阵乘法（需要调用PyTorch的matrix multiply命令）
q = x.mm(y.t())
q

tensor([[2.1021, 2.1021, 2.1021, 2.1021, 2.1021],
        [1.0960, 1.0960, 1.0960, 1.0960, 1.0960],
        [0.6759, 0.6759, 0.6759, 0.6759, 0.6759],
        [1.3870, 1.3870, 1.3870, 1.3870, 1.3870],
        [1.1147, 1.1147, 1.1147, 1.1147, 1.1147]])

### 1.4. 张量与NumPy数组之间的转换

In [10]:
import numpy as np

# x_tensor是一个尺寸为(2, 3)的随机向量
x_tensor = torch.randn(2, 3)
# y_numpy是一个尺寸为(2, 3)的随机矩阵
y_numpy = np.random.randn(2, 3)

In [11]:
# 将张量转换为NumPy
x_numpy = x_tensor.numpy()
x_numpy

array([[-2.1544058 ,  1.4061775 ,  1.0477844 ],
       [-0.7752569 ,  0.87125534, -0.78328764]], dtype=float32)

In [12]:
# 将NumPy转换为张量
y_tensor = torch.from_numpy(y_numpy)
y_tensor

tensor([[ 0.9592, -0.1698, -0.8314],
        [ 0.9617,  0.3688, -0.7929]], dtype=torch.float64)

### 1.5. GPU上的张量运算

In [13]:
torch.cuda.is_available()

False

## 2. 动态计算图实例

In [15]:
# 导入PyTorch中自动微分变量的包
from torch.autograd import Variable
# 创建一个叫作x的自动微分变量，它包含了一个尺寸为(2, 2)的张量，取值为1
x = Variable(torch.ones(2, 2), requires_grad=True)
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [16]:
y = x + 2
y

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)

In [17]:
# 通过y.data查看y中存储的数值
y.data

tensor([[3., 3.],
        [3., 3.]])

In [18]:
# 查看y的grad_fn（其实就是计算图中的箭头）
y.grad_fn

In [19]:
# 再进行运算y * y
z = y * y
# 通过z.grad_fn查看Variable z的grad_fn属性
z.grad_fn

In [20]:
# 再加一步计算
# 对z求平均，即对矩阵的每个元素求和再除以元素的个数
t = torch.mean(z)
t

tensor(9., grad_fn=<MeanBackward0>)

## 3. 自动微分与梯度计算
上述计算过程所代表的函数：t(x) = mean((x + 2)^2)

叶节点x就是自变量，而根节点t就是因变量，mean表示取平均值。如果x发生了变化，那么t也会发生变化。

如果我们观察到t的小变化是△t，那么x的变化是△x是多大呢？

In [21]:
# 通过.backward()进行自动求导计算
t.backward()

In [22]:
print(z.grad)
print(y.grad)
# 只有计算图上的叶节点才可以通过.backward()获得梯度信息
# z和y不是叶节点，所以都没有梯度信息
print(x.grad)

None
None
tensor([[1.5000, 1.5000],
        [1.5000, 1.5000]])


<ipython-input-22-9602199f42dc>:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  print(z.grad)
<ipython-input-22-9602199f42dc>:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  print(y.grad)


In [24]:
# 创建一个1 * 2的自动微分变量（1维向量）s
s = Variable(torch.FloatTensor([[0.01, 0.02]]), requires_grad = True)
# 再创建一个2 * 2的矩阵型自动微分变量x
x = Variable(torch.ones(2, 2), requires_grad = True)

In [25]:
# 用s反复乘以x（矩阵乘法）10次，注意s始终是自动微分变量
for i in range(10):
    s = s.mm(x)
# 对s中的各个元素求均值，得到一个尺寸为（1，1）的张量
z = torch.mean(s)

In [26]:
# 计算叶子节点变量的梯度信息
z.backward()
print(x.grad)
print(s.grad)

tensor([[37.1200, 37.1200],
        [39.6800, 39.6800]])
None


<ipython-input-26-2618903b13bf>:4: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  print(s.grad)
